In [1]:
import pandas as pd
import sys
import os.path as osp

sys.path.append('../src')

In [2]:
data_dir =  "/data/amazon_dataset/"
category =  'Clothing_Shoes_and_Jewelry'
df_w_labels_path = '../outputs/process_label_Clothing_Shoes_and_Jewelry_20211007_074717/df_w_labels.pkl'

# Load cf vectors
cf_vector_path = '../outputs/train_recommender_Clothing_Shoes_and_Jewelry_20211006_122457/emb_df.pkl'
cf_df = pd.read_pickle(cf_vector_path)

# Load df with labels
label_df = pd.read_pickle(df_w_labels_path)


# Load datalaoder

In [6]:
label_df

,asin,img_path,categories,merged_labels,label_vec
1,0000031887,/data/amazon_dataset/Clothing_Shoes_and_Jewelr...,"[[Clothing, Shoes & Jewelry, Girls, Clothing, ...","[Clothing, Shoes & Jewelry + Girls + Clothing]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,0123456479,/data/amazon_dataset/Clothing_Shoes_and_Jewelr...,"[[Clothing, Shoes & Jewelry, Novelty, Costumes...","[Clothing, Shoes & Jewelry + Novelty, Costumes...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
36,1608299953,/data/amazon_dataset/Clothing_Shoes_and_Jewelr...,"[[Clothing, Shoes & Jewelry, Luggage & Travel ...","[Clothing, Shoes & Jewelry + Luggage & Travel ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
47,1617160377,/data/amazon_dataset/Clothing_Shoes_and_Jewelr...,"[[Clothing, Shoes & Jewelry, Luggage & Travel ...","[Clothing, Shoes & Jewelry + Luggage & Travel ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
267,B00001WRHJ,/data/amazon_dataset/Clothing_Shoes_and_Jewelr...,"[[Clothing, Shoes & Jewelry, Boys], [Clothing,...","[Clothing, Shoes & Jewelry + Boys, Clothing, S...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...
1498773,B00KA602SY,/data/amazon_dataset/Clothing_Shoes_and_Jewelr...,"[[Clothing, Shoes & Jewelry, Women, Clothing, ...","[Clothing, Shoes & Jewelry + Women + Clothing ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1499617,B00KCWMG5S,/data/amazon_dataset/Clothing_Shoes_and_Jewelr...,"[[Clothing, Shoes & Jewelry, Women, Clothing, ...","[Clothing, Shoes & Jewelry + Women + Clothing ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1500055,B00KF9180W,/data/amazon_dataset/Clothing_Shoes_and_Jewelr...,"[[Clothing, Shoes & Jewelry, Men, Accessories,...","[Clothing, Shoes & Jewelry + Men + Accessories...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1500301,B00KGCLROK,/data/amazon_dataset/Clothing_Shoes_and_Jewelr...,"[[Sports & Outdoors, Exercise & Fitness, Yoga,...","[Clothing, Shoes & Jewelry + Women + Clothing ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [3]:
meta_df['item_id'] = meta_df['asin'].astype(str)
cf_df['item_id'] = cf_df['item_id'].astype(str)
meta_df['img_path'] = meta_df['imUrl'].apply(lambda x : osp.join(data_dir,category, osp.basename(str(x))))
df = meta_df[['item_id', 'img_path', 'categories']].merge(cf_df, how='inner', on='item_id')
print(len(df), len(meta_df), len(cf_df))

23032 1503384 23032


In [4]:
df['img_exists'] = [osp.exists(path) for path in df['img_path']]
df = df[df['img_exists'] == True]

In [5]:
def process_categories(category_list):
    category_processed_list =[]
    for cat in category_list:
        if cat[0] == 'Clothing, Shoes & Jewelry':
            category_processed_list.append(' + '.join(cat))
    return category_processed_list

In [6]:
df['categories_proc'] = df['categories'].apply(process_categories)
mlb = MultiLabelBinarizer()
df['label_vec'] = mlb.fit_transform(df['categories_proc'].tolist()).tolist()
df.head()


,item_id,img_path,categories,embs,img_exists,categories_proc,label_vec
0,0000031887,/data/amazon_dataset/Clothing_Shoes_and_Jewelr...,"[[Clothing, Shoes & Jewelry, Girls, Clothing, ...","[0.14030873775482178, 0.45290786027908325, -0....",True,"[Clothing, Shoes & Jewelry + Girls + Clothing ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,0123456479,/data/amazon_dataset/Clothing_Shoes_and_Jewelr...,"[[Clothing, Shoes & Jewelry, Novelty, Costumes...","[-0.0495118722319603, 0.3490423262119293, 0.41...",True,"[Clothing, Shoes & Jewelry + Novelty, Costumes...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,1608299953,/data/amazon_dataset/Clothing_Shoes_and_Jewelr...,"[[Clothing, Shoes & Jewelry, Luggage & Travel ...","[0.05297519639134407, 0.33131709694862366, 0.3...",True,"[Clothing, Shoes & Jewelry + Luggage & Travel ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,1617160377,/data/amazon_dataset/Clothing_Shoes_and_Jewelr...,"[[Clothing, Shoes & Jewelry, Luggage & Travel ...","[0.17767387628555298, 0.06136820837855339, -0....",True,"[Clothing, Shoes & Jewelry + Luggage & Travel ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,B00001WRHJ,/data/amazon_dataset/Clothing_Shoes_and_Jewelr...,"[[Clothing, Shoes & Jewelry, Boys], [Clothing,...","[0.13946492969989777, 0.3011769652366638, 0.12...",True,"[Clothing, Shoes & Jewelry + Boys, Clothing, S...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [58]:
from itertools import chain
label_count = pd.value_counts(list(chain.from_iterable(df['categories_proc'].tolist())))
label_count_threshold = 10
print(f'{len(label_count)=} {(label_count<label_count_threshold).sum()=}')

len(label_count)=1776 (label_count<label_count_threshold).sum()=1085


In [32]:
categories = list(label_count[label_count> label_count_threshold].index)

In [55]:
cat_mapper = {cat: ' + '.join(cat.split(' + ')[:-1]) for cat in label_count[label_count<= label_count_threshold].index}

def levelup_category(cats,cat_mapper):
    outs =[]
    for cat in cats:
        if cat in cat_mapper.keys():
            outs.append(cat_mapper[cat])
        else:
            outs.append(cat)
    return outs

df['categories_proc_2'] = df['categories_proc'].apply(lambda cats:  levelup_category(cats,cat_mapper=cat_mapper))
    

In [59]:
from itertools import chain
label_count = pd.value_counts(list(chain.from_iterable(df['categories_proc_2'].tolist())))
print(f'{len(label_count)=} {(label_count<label_count_threshold).sum()=}')

len(label_count)=956 (label_count<label_count_threshold).sum()=180


In [60]:
label_count[label_count<label_count_threshold]

Clothing, Shoes & Jewelry + Novelty, Costumes & More + Work Wear & Uniforms + Clothing + Work Utility & Safety + Women    9
Clothing, Shoes & Jewelry + Women + Clothing + Shorts                                                                     9
Clothing, Shoes & Jewelry + Women + Plus-Size + Socks & Hosiery                                                           9
Clothing, Shoes & Jewelry + Men + Jewelry + Religious                                                                     9
Clothing, Shoes & Jewelry + Women + Clothing + Swimsuits & Cover Ups                                                      9
                                                                                                                         ..
Clothing, Shoes & Jewelry + Women + Jewelry + Fine + Charms & Charm Bracelets + Charms                                    1
Clothing, Shoes & Jewelry + 0-9                                                                                           1
Clothing

In [35]:
mlb = MultiLabelBinarizer()
mlb.fit([categories])

# def is_in_categories(cat_list):
#     for cat in cat_list:
#         mlb.transform([cat]) == 0

# df['label_vec'] = mlb.fit_transform(df['categories_proc'].tolist()).tolist()
# df.head()

MultiLabelBinarizer()

In [39]:
(mlb.transform([df['categories_proc'].iloc[0]]) == 1).sum()

/home/ubuntu/anaconda3/envs/cactus/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:870: UserWarning: unknown class(es) ['Clothing, Shoes & Jewelry + Girls + Clothing + Active + Active Skirts'] will be ignored
  warnings.warn(


0

In [45]:
label_count['Clothing, Shoes & Jewelry + Girls + Clothing']

6